In [1]:
import pandas as pd
import os

def find_bug(df):
    """
    df_offers имеет странные дублирующиеся model_id_pred. Векроятно это баг в функции top_n_similar 
    (не находит достаточное количество до n шт матчей + дальнейший convert_columns_to_int)
    """
    columns_to_check = ['model_id_pred_1', 'model_id_pred_2', 'model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
    duplicate_rows_indices = df[df[columns_to_check].apply(lambda row: row.duplicated().sum() >= 1, axis=1)].index
    new_df = df.loc[duplicate_rows_indices]
    return new_df

def merge_columns_to_list(df, n, column_name):
    """
    Функция для объединения значений из нескольких колонок в один список.
    
    Параметры:
        df (DataFrame): Исходный DataFrame.
        n (int): Количество колонок, которые нужно объединить.
        column_name (str): Имя обрабатываемых колонок.
        
    Возвращает:
        DataFrame: DataFrame с добавленным столбцом, содержащим объединенные значения в виде списка.
    """
    # Создаем список колонок для объединения
    columns = [f"{column_name}_{i}" for i in range(1, n+1)]
    
    # Объединяем значения в список
    df[column_name] = df[columns].apply(lambda row: row.tolist(), axis=1)
    
    return df

def exclude_etalon_from_df(df, df_etalon, imya_kategorii):
    df_exclude = df_etalon[df_etalon.category_name == imya_kategorii]
    print(f"{imya_kategorii}: exclude {len(df_exclude)} rows")
    return df[~df.name.isin(df_exclude.name)]

pth_models = "/home/sondors/Documents/price/ColBERT_data/18_categories/test/models_18_categories.csv"
pth_offers = "/home/sondors/Documents/price/ColBERT/accuracies/offers_and_models/colbert-4446_offers_top_n.csv"
# pth_csv_exclude = '/home/sondors/Documents/price/offers_clusterization/data/14_categories_balanced.csv'

id_category = {
    3902: 'диктофоны, портативные рекордеры',
    510402: 'электронные книги',
    4302: 'автомобильные телевизоры, мониторы',
    2815: 'смарт-часы и браслеты',
    3901: 'портативные медиаплееры',
    3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    510401: 'планшетные компьютеры и мини-планшеты',
    2102: 'наушники, гарнитуры, наушники c микрофоном',
    3903: 'радиоприемники, радиобудильники, радиочасы',
    3907: 'магнитолы',
    280801: 'GPS-навигаторы'
    }

df_models = pd.read_csv(pth_models, sep=";")
# df_models = df_models.drop(columns=['average_price', 'comment'])
df_offers = pd.read_csv(pth_offers, sep=";")
# df_offers = df_offers.drop(columns=['true_match', 'false_match'])

# df_exclude = pd.read_csv(pth_csv_exclude, sep=';')
# for category in id_category.values():
#     df_new = exclude_etalon_from_df(df_offers, df_exclude, category)

df_offers = merge_columns_to_list(df_offers, 5, "model_id_pred")
df_offers = merge_columns_to_list(df_offers, 5, "similarity")
df_offers


,price,name,attrs,description,model_id,external_category,external_brand,external_type,category_name,category_id,...,model_id_pred_2,similarity_2,model_id_pred_3,similarity_3,model_id_pred_4,similarity_4,model_id_pred_5,similarity_5,model_id_pred,similarity
0,1890.0,Очки шлем виртуальной реальности с наушниками ...,"[{'id': 23412, 'type': 'e', 'value': 63357.0},...",Это самое лучшее предложение на существующем р...,3025579,Виртуальная реальность,NaN,NaN,"VR-гарнитуры (VR-очки, шлемы, очки виртуальной...",3908,...,4570782,26.25,3025579,26.08,3025637,25.91,4570781,25.78,"[3025644, 4570782, 3025579, 3025637, 4570781]","[26.94, 26.25, 26.08, 25.91, 25.78]"
1,2500.0,BoboVR Z4,"[{'id': 23412, 'type': 'e', 'value': 63356.0},...",Очки виртуальной реальности для смартфонов с в...,3025579,Очки виртуальной реальности :: Bobovr,BoboVR,NaN,"VR-гарнитуры (VR-очки, шлемы, очки виртуальной...",3908,...,3025580,29.98,3025579,29.70,3025579,29.66,3025579,29.52,"[3025579, 3025580, 3025579, 3025579, 3025579]","[30.97, 29.98, 29.7, 29.66, 29.52]"
2,2845.0,Очки виртуальной реальности BoboVR Z4,"[{'id': 23412, 'type': 'e', 'value': 63356.0}]",Описание товара Виртуальная реальность реальна...,3025579,Все товары :: Электроника :: Портативная техни...,BOBOVR,NaN,"VR-гарнитуры (VR-очки, шлемы, очки виртуальной...",3908,...,3025580,31.27,3025579,31.23,3025580,31.12,3025580,30.92,"[3025579, 3025580, 3025579, 3025580, 3025580]","[31.33, 31.27, 31.23, 31.12, 30.92]"
3,2299.0,Очки виртуальной реальности BOBOVR Z4 / Белые,"[{'id': 23412, 'type': 'e', 'value': 63356.0}]",NaN,3025579,Все товары :: Электроника :: Портативная техни...,BOBOVR,Очки виртуальной реальности,"VR-гарнитуры (VR-очки, шлемы, очки виртуальной...",3908,...,3025580,31.27,3025579,31.23,3025580,31.12,3025580,30.92,"[3025579, 3025580, 3025579, 3025580, 3025580]","[31.33, 31.27, 31.23, 31.12, 30.92]"
4,3800.0,BoboVR Очки BoboVr Z4 + джойстик/пульт управле...,"[{'id': 23412, 'type': 'e', 'value': 63356.0},...",600003652857,3025579,Электроника :: Компьютеры и комплектующие :: А...,BOBOVR,NaN,"VR-гарнитуры (VR-очки, шлемы, очки виртуальной...",3908,...,3025579,29.23,3025579,29.12,3025580,29.05,3025579,28.97,"[3025579, 3025579, 3025579, 3025580, 3025579]","[29.67, 29.23, 29.12, 29.05, 28.97]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177295,5800.0,Наушники Monster Clarity 101 Airlinks MH21902 ...,"[{'id': 9163, 'type': 'b', 'value': 1.0}]",NaN,6200762,"Наушники и усилители, гарнитуры и аксессуары :...",Monster,Наушники,"наушники, гарнитуры, наушники c микрофоном",2102,...,6200762,30.20,6200762,26.20,6200762,26.14,6200762,26.11,"[6200762, 6200762, 6200762, 6200762, 6200762]","[30.36, 30.2, 26.2, 26.14, 26.11]"
177296,6990.0,Беспроводные наушники Monster Clarity 101 Airl...,[],NaN,6200762,Все товары :: Электроника :: Портативная техни...,Monster,Беспроводные наушники,"наушники, гарнитуры, наушники c микрофоном",2102,...,6200762,30.20,6200762,30.16,6200762,30.16,6200762,29.95,"[6200762, 6200762, 6200762, 6200762, 6200762]","[30.7, 30.2, 30.16, 30.16, 29.95]"
177297,3990.0,Наушники Monster Clarity 101 Airlinks MH21902 ...,"[{'id': 9170, 'type': 'b', 'value': 1.0}]",Основные характеристики Тип наушников: вакуумн...,6200762,Смартфоны и гаджеты :: Наушники,Monster,Наушники,"наушники, гарнитуры, наушники c микрофоном",2102,...,6200762,30.27,6200762,27.12,6200762,27.05,6200762,26.91,"[6200762, 6200762, 6200762, 6200762, 6200762]","[30.55, 30.27, 27.12, 27.05, 26.91]"
177298,7190.0,Беспроводные наушники Monster Clarity 101 Airl...,"[{'id': 9163, 'type': 'b', 'value': 1.0}]",Наушники Monster Bluetooth помогут вам остават...,6200762,Наушники :: Беспроводные наушники,Monster,Беспроводные наушники,"наушники, гарнитуры, наушники c микрофоном",2102,...,6200762,30.33,6200762,28.12,6200762,28.09,6200762,27.98,"[6200762, 6200762, 6200762, 6200762, 6200762]","[31.12, 30.33, 28.12, 28.09, 27.98]"


In [2]:
def accuracy_top_n(offer_model_id, offer_model_id_pred, top_k):
    correct_predictions = 0

    for true_id, predicted_ids in zip(offer_model_id, offer_model_id_pred):
        top_k_predictions = predicted_ids[:top_k]
        if true_id in top_k_predictions:
            correct_predictions += 1

    accuracy = correct_predictions / len(offer_model_id)
    return accuracy

def calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k): # проверить функцию
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for true_id, predicted_ids in zip(offer_model_id, offer_model_id_pred):
        top_k_predictions = predicted_ids[:top_k]
        
        if true_id in top_k_predictions:
            true_positives += 1
        else:
            false_negatives += 1
            false_positives += (true_id in predicted_ids)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    return precision, recall

def calc_acc_for_category(df_offers, df_models, imya_kategorii):
    print(f"{imya_kategorii}")
    df_offers = df_offers[df_offers.category_name == imya_kategorii]
    df_models = df_models[df_models.category_name == imya_kategorii]

    offer_names = list(df_offers['name'])
    offer_model_id = list(df_offers['model_id'])
    offer_model_id_pred = list(df_offers['model_id_pred'])

    model_full_names = list(df_models['full_name'])

    print(f"\tофферов {len(offer_model_id)} шт поиск матча для каждого среди {len(model_full_names)} шт моделей:\n")

    top1_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 1)
    top2_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 2)
    top3_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 3)
    top4_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 4)
    top5_accuracy = accuracy_top_n(offer_model_id, offer_model_id_pred, 5)

    top1_precision, top1_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=1)
    top3_precision, top3_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=3)
    top5_precision, top5_recall = calculate_precision_recall(offer_model_id, offer_model_id_pred, top_k=5)

    print(f'\tTop-1: Accuracy = {top1_accuracy:.2%}')
    print(f'\tTop-3: Accuracy = {top3_accuracy:.2%}')
    print(f'\tTop-5: Accuracy = {top5_accuracy:.2%}\n')

    # print(f'\tTop-1: Accuracy = {top1_accuracy:.2%}, Precision = {top1_precision:.2%}, Recall = {top1_recall:.2%}')
    # print(f'\tTop-3: Accuracy = {top3_accuracy:.2%}, Precision = {top3_precision:.2%}, Recall = {top3_recall:.2%}')
    # print(f'\tTop-5: Accuracy = {top5_accuracy:.2%}, Precision = {top5_precision:.2%}, Recall = {top5_recall:.2%}\n')
    return (top1_accuracy, top2_accuracy, top3_accuracy, top4_accuracy, top5_accuracy)

for category in id_category.values():
    accuracies_top_n = calc_acc_for_category(df_offers, df_models, category)

диктофоны, портативные рекордеры
	офферов 710 шт поиск матча для каждого среди 488 шт моделей:

	Top-1: Accuracy = 93.80%
	Top-3: Accuracy = 94.93%
	Top-5: Accuracy = 95.49%

электронные книги
	офферов 760 шт поиск матча для каждого среди 680 шт моделей:

	Top-1: Accuracy = 93.82%
	Top-3: Accuracy = 96.58%
	Top-5: Accuracy = 97.24%

автомобильные телевизоры, мониторы
	офферов 280 шт поиск матча для каждого среди 790 шт моделей:

	Top-1: Accuracy = 60.71%
	Top-3: Accuracy = 67.50%
	Top-5: Accuracy = 70.36%

смарт-часы и браслеты
	офферов 11210 шт поиск матча для каждого среди 2577 шт моделей:

	Top-1: Accuracy = 92.07%
	Top-3: Accuracy = 94.76%
	Top-5: Accuracy = 95.93%

портативные медиаплееры
	офферов 1410 шт поиск матча для каждого среди 847 шт моделей:

	Top-1: Accuracy = 93.33%
	Top-3: Accuracy = 97.16%
	Top-5: Accuracy = 97.73%

портативная акустика
	офферов 15610 шт поиск матча для каждого среди 3800 шт моделей:

	Top-1: Accuracy = 89.33%
	Top-3: Accuracy = 91.89%
	Top-5: Accurac

In [3]:
def write_errors(df_offers, df, df_models, dst_fld, id_category):
    os.makedirs(dst_fld, exist_ok=True)
    for category in id_category.values():
        category_error = df[df.category_name == category]
        category_all = df_offers[df_offers.category_name == category]
        accuracies_top_n = calc_acc_for_category(category_error, df_models, category)
        with open(os.path.join(dst_fld, f"{category}.txt"), "a") as txt:
            txt.write(f'Всего офферов {len(category_all)} шт из них ошибок {len(category_error)} шт\nРаспределение ошибок:\n')
            txt.write(f'\nTop-1: Accuracy = {accuracies_top_n[0]:.2%}')
            txt.write(f'\nTop-2: Accuracy = {accuracies_top_n[1]:.2%}')
            txt.write(f'\nTop-3: Accuracy = {accuracies_top_n[2]:.2%}')
            txt.write(f'\nTop-4: Accuracy = {accuracies_top_n[3]:.2%}')
            txt.write(f'\nTop-5: Accuracy = {accuracies_top_n[4]:.2%}\n\n')
            txt.write("Истинная модель:\nПоле name оффера --> Найденная модель топ1")
            txt.write("\n____________________________________________________________\n")
            for index, row in category_error.iterrows():
                model_gt = list(df_models[df_models.model_id == row['model_id']]['full_name'])[0]
                model_pred = list(df_models[df_models.model_id == row['model_id_pred_1']]['full_name'])[0]
                txt.write(f"\n{model_gt}:\n{row['name']} --> {model_pred}\n")                
def exclude_from_df(df, col):
    return df[df['model_id'] != df[col]].reset_index(drop=True)

# clms = ['model_id_pred_1','model_id_pred_2','model_id_pred_3', 'model_id_pred_4', 'model_id_pred_5']
# df = df_offers.copy()
# for i in range(len(clms)):
#     df = exclude_from_df(df, clms[i])
#     dst_fld = f"/home/sondors/Documents/price/ColBERT/EVAL/errors_ColBERT/exclude_top_{i+1}"
#     write_errors(df_offers, df, df_models, dst_fld, id_category)